In [1]:
import numpy as np
%matplotlib inline

これまで我々が見てきたNNは，フィードフォワードと呼ばれるネットワークで，流れが1方向でしかなかった.  
フィードフォワードでは時系列データをうまく扱うことができない．  
そこでリカレントニューラルネットワーク(RNN)の出番である．  
本章では，フィードフォワードの問題点を指摘し，RNNがその問題を解決することを示す．

# 確率と言語モデル

## word2vecを確率の視点から眺める

コーパス$w_1, w_2, ... w_T$が与えられているとき，$w_t$がターゲットとなる確率は，コンテキスト$w_{t-1}, w{t+1}$を使って
$$ P(w_t | w_{t-1}, w_{t+1})$$
と書ける．  
ここで，コンテキストの窓を非対称にして全て左側にコンテキストがあるとすると，$w_t$がターゲットとなる確率は
$$ P(w_t | w_{t-2}, w_{t-1})$$
このとき，CBOWモデルが扱う損失関数は
$$ L=-\log P(w_t|w_{t-2}, w_{t-1}) $$

## 言語モデル
言語モデルは，単語の並びがどれだけ自然であるかを確率で評価する．  
例えば，
- you say goodbye -> 0.092
- you say good die -> 0.000000000000032  

$ w_1, ..., w_m $ という順序で単語が出現する確率は，同時確率 $ P(w_1, ..., w_m) $ で表される．  
これを事後確率と確率の乗法定理 $P(A, B) = P(A|B)P(B)$ を使って分解すると　　
$$ P(w_1, ..., w_m) = P(w_m|w_1, ... w_{m-1})P(w_{m-1}|w_1, ... w_{m-2}) ... P(w_3|w_1, w_2)P(w_2|w_1)P(w_1)$$  
$$ = \prod_{t=1}^{m} P(w_t|w_1, ..., w_{t-1})$$

確率の乗法定理は，「AとBが両方同時に起こる確率 $P(A,B)$」は，「Bが起こる確率$P(B)$」と「Bが起こったあとにAが起きる確率P(A|B)」を掛け合わせたものである． 
また，$P(A, B) = P(B|A)P(A)$と書くこともできる．  
ここで注目すべきは，事後確率が対象の単語より左の全ての単語をコンテキストとした時の確率ということである．  
また， $P(w_t|w_1, ... w_{t-1})$を表すモデルは，条件付き言語モデルと呼ばれる．これを言語モデルと呼ぶ場合も多く見られる．

## CBOWモデルを言語モデルに？
CBOWモデルを無理やり言語モデルに適用するには，コンテキストのサイズをある値に限定することで近似的に表すことができる．  
$$ P(w_1, ..., w_m) = \prod_{t=1}^{m} P(w_t|w_1, ..., w_{t-1}) \approx \prod_{t=1}^{m} P(w_t|w_{t-2}, w_{t-1}) $$
  
マルコフ性  
未来の状態が現在の状態だけに依存して決まること．  
ここで，ある事象の確率がその直前のN個の事象だけに依存するとき，これを「N階マルコフ連鎖」という． 　
今は直前の2つに依存して次の単語が決まるので，2階マルコフ連鎖と呼べる．  
  
コンテキストのサイズは任意に設定できるが，固定する必要があるところに問題がある．  
例えば，コンテキストのサイズが10だが答えとなるTomなどの固有名詞が18個前にしかない時，この推論問題に答えることはできない．  
コンテキストを20にしたりすれば答えることはできる．  
  
しかし次にはコンテキスト内の単語の並びが無視されるという問題がある．  
例えば，(you,say)というコンテキストと(say,you)というコンテキストが同じものとして表される．  
これは，CBOWモデルの中間層を各コンテキストが共有しているために起きる．  
そこで，コンテキストごとに中間層を設けることで，すなわち複数の中間層を「連結(contcatenate)」することでこの問題に対処できる．  
しかし，そのようにするとコンテキストのサイズに比例して重みパラメータが増大してしまう．  

これらの問題を解決するのがRNNである．  
RNNは，コンテキストがどれだけ長くても，そのコンテキストの情報を記憶するメカニズムを持つ．  
  
ちなみに，実はword2vecの方が後に提案されている．  
RNNによる言語モデルでも単語の分散表現を獲得できるのだが，新たな語彙の追加しやすさや単語の分散表現の質の向上のためword2vecが提案された．

# RNNとは
Recurrent Neural Networkは日本語では「再起ニューラルネットワーク」や「循環ニューラルネットワーク」と呼ばれる．  
これに対してRecursive Neural Networkというものもあるが，こちらは主に木構造のデータを処理するためのネットワークで，RNNとは別物である．

## 循環するニューラルネットワーク
RNNの特徴は，閉路を持つことである．  
入力データを$(x_0, x_1, ... x_t, ...)$として， 出力データ$(h_0, h_1, ... h_t, ...)$ を自分にも入力する，すなわち閉路を持つ層をRNNレイヤと名付ける．　　
ここで，　$x_t$や$h_t$はベクトルを想定する．例えば，ある単語の分散表現を$x_t$としたりする．  
また，これまではデータが左から右へ流れていたが，以降は左右方向にレイヤが展開されるため，紙面の都合上，下から上へデータが流れるように描画される．  

## ループの展開
RNNレイヤは自身に出力データを流していたが，これを同じレイヤの別のRNNレイヤに流すことで，ループを展開する．  
左から右へ，同じレイヤのRNNレイヤが並び，その順が時系列の順番になっている．  
その出力が，左から$h_0, h_1, ... h_t$となる．  
「t番目の単語」や「t番目のRNNレイヤ」は，「時刻tの単語」や「時刻tのRNNレイヤ」というように，「時刻」という言葉でも表される．  
このとき，$h_t$は以下の計算で算出される．  
$$ h_t = \tanh (h_{t-1} W_h + x_t W_x + b) $$
RNNレイヤは重みを2つ持つ．  
入力$x$への重み$W_x$と一つ前のRNNの出力$h_{t-1}$に対する重み$W_h$である．  
これらの和にさらにバイアス$b$を加え， 活性化関数としてtanhを適用したものがRNNレイヤの出力になっている．  
  
この式では，RNNが$h_t$という現在の出力を状態として保持していて，上記の式で$h_t$が更新されていると見ることもできる．  
そのため，RNNレイヤは状態を持つレイヤやメモリを持つレイヤと呼ばれている．  
  
この$h_t$は「隠れ状態」や「隠れ状態ベクトル」と呼ばれている．  
  
多くの文献では，RNNレイヤの次の時刻への矢印はRNNレイヤから生えているが，本書では同じデータがコピーして分岐されたものであることを明示的に示すため，RNNレイヤの次の層への矢印から次の自国への矢印を伸ばすことにする．

## Backpropagation Through Time
ループを展開した後のRNNは，時間方向と逆方向に誤差逆伝播法を使って重みを更新していくことができる．  
このようなRNNにおける誤差逆伝播ん法をBackpropagation Through Time, 略してBPTTと呼ぶ.  
  
しかし，RNNレイヤを長く繋げると計算リソース(RNNが使うメモリ)が膨大になり，逆伝播の勾配も不安定になってしまう．　　

## Truncated BPTT
時間方向に長くなりすぎたネットワークを適当な場所で切り取って(truncate)小さなネットワークを複数作ること．  
これら小さくなったネットワークに対して，誤差逆伝播法を行う．  
正確には，ネットワークの逆伝播の時だけつながりを断ち切る．順伝播はそのまま行う．  
  
具体例を見ていく.  
1000個の長さのコーパス$x_0, x_1, ... x_{999}$があるとする． このコーパスは，複数の文が連結されたものであるとする．  
Truncated BPTTをするため，逆伝播のときは1000個連結されたRNNを「ブロック」に切り分ける．  
ここでは，それぞれのブロックのRNNのレイヤの長さが10になるようにする．  これによって，それぞれのブロックではそれより未来のデータについて考えなくていい．  

順伝播の時は，データを順番に(シーケンシャルに)与える必要がある．  
これまでミニバッチ学習ではデータをランダムに選んでいたが，Truncated BPTTでは順伝播の時にデータをシーケンシャルに与える必要がある．  
  
上の例でいうと，まずは一つ目のブロックの入力データ$x_0, ... x_9$をRNNレイヤに与え, forwardとbackwardを行う．  
次は，入力データ$x_{10}, ... ,x_{19}$をRNNに与え, forwardとbackwardを行う．  
このとき，前のブロックの出力$h_9$をこのブロックの先頭のRNNに与える必要があることに注意する．これによって順伝播のつながりは維持できる.  
以上のようにして順伝播とTruncated BPTTを進めていく．  

## Truncated BPTTのミニバッチ学習
RNNでTruncated BPTTする場合のミニバッチ学習では，開始位置をずらしたデータを利用する．  
これによって，各単語の前後関係を保ったまま，つまりシーケンシャルなまま，違うデータをRNNに与えることができる．  

例えば上の例では$x_0$からデータを与えていったが，別のデータでは$x_500$からデータを与えていく．  
なお，途中で終端に達した場合は，先頭に戻る．  

# RNNの実装
横方向にRNNをT個だけ繋げたものを一つの層として実装する．  
この層は，シーケンシャルなデータ$xs = (x_0, x_1, ... x_{T-1})$を入力すると，隠れ層へ$hs=(h_0, h_1, ..., h_{T-1})$を出力する．  
このひとまとまりにした層を「Time RNN」レイヤと呼ぶ．↔️ 1ステップの処理を行う「RNNレイヤ」  
後ほどTime Affine レイヤやTime Embeddingレイヤといった，時系列データをまとめて処理するレイヤも登場する．  

まず1ステップの処理を行うRNNクラスを作成し，Tステップの処理をまとめて行うレイヤをTimeRNNクラスとして完成させる．  

## RNNレイヤの実装
RNNレイヤの計算式を再掲
$$ h_t = \tanh (h_{t-1} W_h + x_t W_x + b) $$
$x_t$と$h_{t-1}$はミニバッチ処理のため，$N$個のデータを持つとする．各行がデータで，それらが列方向に並ぶ．  
そのため，$x_t$と$h_{t-1}$はそれぞれ$N \times H $, $N \times D$の行列になる．  
ここで，$H$は隠れ状態ベクトルの次元数， $D$は入力データの次元数である．  
RNNレイヤの計算では，次元数が以下のようになることに注意するのが重要である．

$$ h_{t-1} W_h + x_t W_x + b \Rightarrow h_t$$
$$ (N \times H) (H\times H) + (N \times D) (D \times H) + (N \times H) \Rightarrow (N \times H) $$  

以下この式とシステムに従いRNNレイヤを実装する．

In [2]:
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b] # 初期重みをparamsに設定
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)] # 初期勾配をgradsに設定
        self.cache = None # BPTT用のキャッシュ変数(メモリ)
    
    def forward(self, x, h_prev): # 入力データと前の時刻の出力
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        
        # tanhの逆伝播
        dt = dh_next * (1 - h_next ** 2)
        
        # 加算ノードはdtをそのまま前のレイヤに返す
        
        # b方向へのrepeatの逆伝播
        db = np.sum(dt, axis=0)
        
        # h_prev方向へのMatMulの逆伝播
        dWh = h_prev.T @ dt
        dh_prev = dt @ Wh.T
        
        # x方向へのMatMulの逆伝播
        dWx = x.T @ dt
        dx = dt @ Wx.T
        
        self.grads[0][...] = dWx # 3点リーダによって，浅いコピー(参照のコピー)ではなく深いコピー(値のコピー)を行う．
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev

ここでちょっと逆伝播の復習

- 加算ノードの逆伝播  
上流から入ってきた勾配$\frac{\partial L}{\partial z}$をそのまま受け流す. 3個以上の加算ノードでも同様にそのまま受け流す．
$$ z = x + y $$
$$ \frac{\partial z}{\partial x} = 1, \frac{\partial z}{\partial y} = 1 $$
上流で発生した損失Lに対するxの勾配
$$　\frac{\partial L}{\partial x} = \frac{\partial L}{\partial z} \cdot \frac{\partial z}{\partial x} = \frac{\partial L}{\partial z} \cdot 1 = \frac{\partial L}{\partial z}$$
上流で発生した損失Lに対するyの勾配
$$ \frac{\partial L}{\partial y} = \frac{\partial L}{\partial z} \cdot \frac{\partial z}{\partial y} = \frac{\partial L}{\partial z} \cdot 1 = \frac{\partial L}{\partial z}$$  

<br><br>
- Repeatノードの逆伝播
RNNレイヤの実装では，N個のミニバッチにそれぞれ同じ重さベクトルbを加算している．  
これは分岐ノードの一般形であるRepeatノードの処理が行われていることになる．  
分岐ノード，Repeatノードでは，上流からやってきた重みを全て足し合わせて下流に返す．  
Repeatノードの逆伝播は，$D\times N$の上流からの勾配をN個の方向に足し合わせ，D次元のベクトルを作り出し，そのベクトルを勾配とする．

<br><br>
- 行列の積(Matrix Multiply: MatMul)の逆伝播  
p30, 31ではシグマを使ってもうちょい厳密に書いているが，自分の理解を書いてみる．まずは結果から
$$ y = xW $$
$$ (N \times H) = (N \times D)(D \times H) $$
$$ \frac{\partial L}{\partial x} = \frac{\partial L}{\partial y}W^{\top}, \frac{\partial L}{\partial W} = x^{\top} \frac{\partial L}{\partial y} $$
$$ (N \times D) = (N \times H)(H \times D), (D \times H) = (D \times N)(N \times H) $$
<br>

例えば，以下のような場合を考える．
$$ y = xW = \left( \begin {array}{ccc} a & b & c \end{array} \right) \left( \begin {array}{ccc} 1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 9 \end{array} \right) = \left( \begin {array}{ccc} 1a+2b+3c & 4a+5b+6c & 7a+8b+9c \end {array} \right) $$

このとき， [ベクトルをベクトルで微分する計算の定義](https://qiita.com/AnchorBlues/items/8fe2483a3a72676eb96d)を使って

$$ \frac{\partial y}{\partial x} = \left( \begin{array}{ccc} \frac{\partial y_1}{\partial a} & \frac{\partial y_2}{\partial a} & \frac{\partial y_3}{\partial a}\\ \frac{\partial y_1}{\partial b} & \frac{\partial y_2}{\partial b} & \frac{\partial y_3}{\partial b} \\ \frac{\partial y_3}{\partial c} & \frac{\partial y_2}{\partial c} & \frac{\partial y_3}{\partial c}\end{array} \right) =\left( \begin{array}{ccc} 1 & 4 & 7 \\ 2 & 5 & 8 \\ 3 & 6 & 9 \end{array} \right) = W^\top$$
つまり，
$$ \frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial x} = \frac{\partial L}{\partial y} W^\top $$ 
$\frac{\partial L}{\partial W}　= x^\top \frac{\partial L}{\partial y}$も同様に求められる．(ベクトル$y$を行列$W$で微分)
<br><br>
- tanhの逆伝播
$$ y = \tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$$
の微分を求める．商の微分法を使って
$$ \frac{\partial \tanh(x)}{\partial x} = \frac {({e^x + e^{-x}})({e^x + e^{-x}}) - ({e^x - e^{-x}})({e^x - e^{-x}})}{({e^x + e^{-x}})^2} $$
$$ = 1 - \Bigl\{ \frac {({e^x - e^{-x}})}{({e^x + e^{-x}})} \Bigr\}^2 $$
$$ = 1 - \tanh(x)^2 = 1 - y^2 $$
よって, 上流からの勾配$\frac{\partial L} {\partial y}$を使って
$$ \frac{\partial L} {\partial x} = \frac{\partial L} {\partial y} \cdot \frac{\partial y} {\partial x} = \frac{\partial L} {\partial y} (1 - y^2) $$
を下流に返せば良い．  
ここでyは順伝播の時に保持しておく．RNNレイヤの実装でいうh_next， 最終的な出力を使うことになる．

## Time RNN レイヤの実装
T個のRNNレイヤで構成されるTime RNNレイヤを実装する．  
Time RNNレイヤは隠れ状態hをメンバ変数として保持し，ブロックに分割した時に次のブロックへhを渡す．  
隠れ状態を引き継ぐかどうかはstatefulという引数で調整する．

In [3]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False): # stateful: 状態を持つ，という意味
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None # 複数のRNNレイヤをリストとして持つ予定
        self.h, self.dh = None, None # forwardの最後の出力(次のブロックへ）を保持，　backwardの最後(先頭)の出力（前のブロックへ)を保持
        self.stateful = stateful # Falseのとき，forwardのたびに最初のRNNレイヤの隠れ状態をゼロ行列で初期化する． (ステートレス)
    
    def set_state(self, h):
        self.h = h
    
    def reset_state(self):
        self.h = None
    
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape # ミニバッチ数，　文章の長さ，　分散表現の次元数(語彙数)
        D, H = Wx.shape # 分散表現の次元数(語彙数), 隠れ層のサイズ
        
        self.layers = []
        hs = np.empty((N, T, H), dtype='f')
        
        if not self.stateful or self.h is None: 
            self.h = np.zeros((N, H), dtype='f')
        
        # 長さTのRNNループを展開
        for t in range(T):
            layer = RNN(*self.params) # ブロック内では同じ重みを使うようだ
            self.h = layer.forward(xs[:, t, :], self.h) # tの初回が別のブロックから渡されるかがstatefulによって決められることになる．
            hs[:, t,  :] = self.h # 全行(ミニバッチ全て), ｔ列(ｔ番目の単語), 全チャネル(単語の分散表現ベクトル全体) の出力をこのRNNレイヤの出力とする
            self.layers.append(layer)
        
        return hs
    
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape
        
        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0] # Wx, Wh, b
        
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh) # 上流からの勾配(dhs)と次の時刻(dh)の勾配を合算して下流(dxs)と前の時刻(dh)に渡す
            dxs[:, t, :] = dx
            
            # Time RNNレイヤの勾配は全てのRNNレイヤの勾配の合計
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
        
        # Time RNNレイヤのgradsに今回のbackwardの勾配をDeepコピー
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        
        # 前の時刻のブロックの全体に渡すhの勾配
        self.dh = dh
    
        return dxs # 下流への勾配

In [4]:
# hs[:, t, :]がよくわからんかったので実験

print("2次元の場合")
A = np.array([[1,2], [3,4], [5,6]]) # コロン選択はnumpy Arrayでないとできない

print(A[0, :], "\n") # 0行全列
print(A[:, 0], "\n") # 全行0列 (列リストになるわけではない．1次元にされる)
print(A[:, 0].reshape(A.shape[0], 1), "\n") #　全行0列を元の形に直す

print("3次元の場合")
N = 5 # ミニバッチ数
T = 2 # 文章(コンテキスト)の長さ
H = 3 # 隠れ層のサイズ

hs = np.random.randn(N, T, H) # 5行 2列 3チャネル
print(hs)

for t in range(T):
    print()
    hs[:, t, :] = np.zeros((N, H), dtype='f') #　全行 ｔ列 全チャネル をゼロに， つまり文章の中の単語ごとの操作
    print(hs)

2次元の場合
[1 2] 

[1 3 5] 

[[1]
 [3]
 [5]] 

3次元の場合
[[[-0.91589124 -1.01238345 -1.02766694]
  [ 0.43228832  0.79576306 -0.51833897]]

 [[ 0.77678142  1.11187406  0.49837476]
  [-0.35936794 -0.75434901  0.60806009]]

 [[-0.14182835  1.12086526 -0.83401287]
  [ 1.89068878 -0.41334254 -1.40600115]]

 [[ 0.34754026  0.85928899  1.35268569]
  [-0.68139318 -1.53836103  0.01915062]]

 [[ 1.05162127  0.38942058  0.57319717]
  [ 0.51110767  1.48095137 -0.04772596]]]

[[[ 0.          0.          0.        ]
  [ 0.43228832  0.79576306 -0.51833897]]

 [[ 0.          0.          0.        ]
  [-0.35936794 -0.75434901  0.60806009]]

 [[ 0.          0.          0.        ]
  [ 1.89068878 -0.41334254 -1.40600115]]

 [[ 0.          0.          0.        ]
  [-0.68139318 -1.53836103  0.01915062]]

 [[ 0.          0.          0.        ]
  [ 0.51110767  1.48095137 -0.04772596]]]

[[[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [

# 時系列データを扱うレイヤの実装
RNNLM: RNN Language Model: RNN言語モデル: RNNによる単語の並びの自然性判定

## RNNLMの全体図
$w_t$ -> Embedding -> RNN -> Affine -> Softmax -> $y_t$となる